# Data Integration
### Table of Contents
- [Requirements](#requirements)
- [Structuring Historical Yield Data](#structuring-historical-yield-data)
- [Structuring Historical Price Received Data](#structuring-historical-price-received-data)
- [Structuring Historical Weather Data](#structuring-historical-weather-data)
- [Integrating Data](#integrating-data)

## Requirements

In [8]:
import pandas as pd

In [9]:
## saving state names for later use
states = ['ILLINOIS', 'INDIANA', 'IOWA', 'MINNESOTA', 'MISSOURI', 'NEBRASKA']
months = ['APR', 'AUG', 'DEC', 'FEB', 'JAN', 'JUL', 'JUN', 'MAR', 'MAY', 'NOV', 'OCT', 'SEP']

## reading raw data CSVs
yield_raw = pd.read_csv('../../data/raw/yield_raw.csv') # file path appears as `data/raw/yield_raw.csv` in `integration.py`
price_received_raw = pd.read_csv('../../data/raw/price_received_raw.csv') # file path appears as `data/raw/price_received_raw.csv` in `integration.py`
weather_raw = pd.read_csv('../../data/raw/weather_raw.csv') # file path appears as `data/raw/weather_raw.csv` in `integration.py`

## Structuring Historical Yield Data

In [10]:
## dropping records of unnecessary states and focusing the reference period to only annual records
## dropping duplicate records and preserving the first appearence of year/state record
yield_raw = yield_raw[
    (yield_raw['state_name'].isin(states)) & (yield_raw['reference_period_desc'] == 'YEAR')
]\
    .drop_duplicates(subset=['year', 'state_name', 'util_practice_desc'])

## pivotting the dataframe
yield_raw = yield_raw\
    .pivot(
    index=['year', 'state_name'],
    columns='util_practice_desc',
    values='Value'
    )\
    .reset_index()

## Structuring Historical Price Received Data

In [11]:
## pivotting the dataframe
price_received_raw = price_received_raw\
    .pivot(
    index=['year', 'state_name'],
    columns='reference_period_desc',
    values='Value'
    )\
    .reset_index()

## renaming columns appropriately
for x in price_received_raw.columns:
    price_received_raw.rename(columns={x:x+'_preceived'}, inplace=True) if x in months else x
    price_received_raw.rename(columns={'MARKETING YEAR':'marketing_year'}, inplace=True) if x not in months else x

## Structuring Historical Weather Data

In [12]:
weather_raw = weather_raw\
    .groupby(['State', 'Year'], as_index=False)\
    .agg(
        temp_mean = ('tavg', 'mean'),
        temp_min = ('tmin', 'min'),
        temp_max = ('tmax', 'max'),
        pcp_sum = ('pcp', 'sum'),
        pdsi_mean = ('pdsi', 'mean')
    )\
    .rename(columns={'State':'state_name', 'Year':'year'})

## Integrating Data

In [13]:
## merge `yield_raw` and `price_received_raw`
temp = yield_raw.merge(price_received_raw, on=['year', 'state_name'], how='outer')

## merge the aforementioned merged dataframe with `weather_raw`
integrated = temp.merge(
    weather_raw,
    on=['year', 'state_name'],
    how='outer'
)

integrated

,year,state_name,GRAIN,SILAGE,APR_preceived,AUG_preceived,DEC_preceived,FEB_preceived,JAN_preceived,JUL_preceived,...,marketing_year,MAY_preceived,NOV_preceived,OCT_preceived,SEP_preceived,temp_mean,temp_min,temp_max,pcp_sum,pdsi_mean
0,1866,ILLINOIS,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1866,INDIANA,36.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1866,IOWA,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1866,MINNESOTA,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1866,MISSOURI,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,2025,INDIANA,205.0,NaN,4.76,NaN,NaN,4.66,4.32,NaN,...,NaN,4.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
956,2025,IOWA,222.0,NaN,4.64,NaN,NaN,4.59,4.37,NaN,...,NaN,4.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
957,2025,MINNESOTA,202.0,NaN,4.48,NaN,NaN,4.47,4.20,NaN,...,NaN,4.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
958,2025,MISSOURI,191.0,NaN,4.63,NaN,NaN,4.56,4.33,NaN,...,NaN,4.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
## merge `yield_raw` and `price_received_raw`
temp = yield_raw.merge(price_received_raw, on=['year', 'state_name'], how='outer')

## merge the aforementioned merged dataframe with `weather_raw`
integrated = temp.merge(
    weather_raw,
    on=['year', 'state_name'],
    how='outer'
)

## making all columns uppercase for consistency
integrated = integrated.rename(columns={col:col.upper() for col in integrated.columns})

integrated = integrated[[
    'STATE_NAME', 'YEAR',
    'GRAIN', 'SILAGE',
    'MARKETING_YEAR', 'TEMP_MEAN', 'TEMP_MIN', 'TEMP_MAX', 'PCP_SUM', 'PDSI_MEAN'
]]\
    .rename(columns={'STATE_NAME':'STATE', 'MARKETING_YEAR':'PRICE_RECEIVED'
})

# save the dataframe as a local CSV
integrated.to_csv('../../data/raw/integrated.csv', index=False) # file path appears as `data/raw/integrated.csv` in `integration.py`